## Imports

Rheology package must be installed on local computer.

In [1]:
import pandas as pd
import rheology as rh
import math

## Load Data
 Load raw data into a pandas dataframe for each sample (n). Make sure all columns in the csv has cell **type "Number" with no , separating 1000.** Also ensure to **delete the notes at the top of the csv with the parameters being the first row** (Meas. Pts., Time, Storage Modulus, etc.).

In [4]:
# PXP 37C data
p1 = pd.read_csv('exampledata/PXP_N1.csv', encoding="ISO-8859-1")
p2 = pd.read_csv('exampledata/PXP_N2.csv', encoding="ISO-8859-1")
p3 = pd.read_csv('exampledata/PXP_N3.csv', encoding="ISO-8859-1")
p4 = pd.read_csv('exampledata/PXP_N4.csv', encoding="ISO-8859-1")

# TXT 37C data
t1 = pd.read_csv('exampledata/TXT_N1.csv', encoding="ISO-8859-1")
t2 = pd.read_csv('exampledata/TXT_N2.csv', encoding="ISO-8859-1")
t3 = pd.read_csv('exampledata/TXT_N3.csv', encoding="ISO-8859-1")
t4 = pd.read_csv('exampledata/TXT_N4.csv', encoding="ISO-8859-1")

Parse the data into each test using `rh.all_tests_n(df)`. This will return a dictionary where each entry corresponds to a separate test in the rheology protocol. See below for the dictionary keys.

0 : time sweep \
1 : frequency sweep \
2 : time sweep \
3 : strain sweep \
4 : time sweep \
5 : cyclic strain sweep \
6 : shear thinning test

In [5]:
pxp = rh.all_tests(p1, p2, p3, p4)
txt = rh.all_tests(t1, t2, t3, t4)

## Average Storage Modulus

In [7]:
sm_pxp = rh.storage_modulus(pxp)
sm_txt = rh.storage_modulus(txt)

name = ["n1 G' [Pa]", "n2 G' [Pa]", "n3 G' [Pa]", "n4 G' [Pa]"]

sm = pd.DataFrame([sm_pxp, sm_txt], columns=name, index=['RGD.PXP.RGD', 'T40A'])
sm['Mean'] = sm.mean(axis=1)
sm

,n1 G' [Pa],n2 G' [Pa],n3 G' [Pa],n4 G' [Pa],Mean
RGD.PXP.RGD,5257.130952,4596.690476,3708.833333,5836.702381,4849.839286
T40A,6865.654762,10835.357143,7292.440476,2527.928571,6880.345238


## Angular Frequency Crossover

In [8]:
name = ['n1 [rad/s]', 'n2 [rad/s]', 'n3 [rad/s]', 'n4 [rad/s]']

aco_pxp = rh.crossover(pxp, name, cotype=2)
aco_pxp = aco_pxp.rename(index={'Frequency Sweep 1': 'RGD.PXP.RGD'})

In [9]:
aco_txt = rh.crossover(txt, name, cotype=2)
aco_txt = aco_txt.rename(index={'Frequency Sweep 1': 'T40A'})

In [10]:
frames = [aco_pxp, aco_txt]
aco = pd.concat(frames)
aco

,n1 [rad/s],n2 [rad/s],n3 [rad/s],n4 [rad/s],Mean
Test,,,,,
RGD.PXP.RGD,12.726049,10.218855,13.346851,6.522532,10.703572
T40A,3.144564,1.572737,1.482496,3.129497,2.332324


## Strain Crossover

In [11]:
name = ['n1 %Strain', 'n2 %Strain', 'n3 %Strain', 'n4 %Strain']

sco_pxp = rh.crossover(pxp, name, cotype=1)
sco_pxp = sco_pxp.rename(index={'Strain Sweep 3': 'RGD.PXP.RGD'})

In [12]:
sco_txt = rh.crossover(txt, name, cotype=1)
sco_txt = sco_txt.rename(index={'Strain Sweep 3': 'T40A'})

In [13]:
frames = [sco_pxp, sco_txt]
sco = pd.concat(frames, sort=False)
sco

,n1 %Strain,n2 %Strain,n3 %Strain,n4 %Strain,Mean
Test,,,,,
RGD.PXP.RGD,156.850272,147.893704,165.471030,140.654508,152.717379
T40A,147.926460,117.934776,136.476612,154.200841,139.134672


## Recovery Time $t_{1/2}$

In [14]:
start = [1081, 1727, 2373, 3019]
name = ['n1 t1/2 [s]', 'n2 t1/2 [s]', 'n3 t1/2 [s]', 'n4 t1/2 [s]']

rth_pxp = rh.recovery(start, pxp, name, rtype=1)
rth_pxp = rth_pxp.rename(index={'Cyclic Strain Sweep 5': 'RGD.PXP.RGD'})

In [15]:
rth_txt = rh.recovery(start, txt, name, rtype=1)
rth_txt = rth_txt.rename(index={'Cyclic Strain Sweep 5': 'T40A'})

In [16]:
frames = [rth_pxp, rth_txt]
rth = pd.concat(frames, sort=False)
rth

,n1 t1/2 [s],n2 t1/2 [s],n3 t1/2 [s],n4 t1/2 [s],Mean
Test,,,,,
RGD.PXP.RGD,33.543675,21.159587,40.149452,34.339929,32.298161
T40A,37.072018,24.220627,46.656151,14.483134,30.607983


## Recovery Time Crossover $t$

In [21]:
name = ['n1 t [s]', 'n2 t [s]', 'n3 t [s]', 'n4 t [s]']

rco_pxp = rh.recovery(start, pxp, name, rtype=2)
rco_pxp = rco_pxp.rename(index={'Cyclic Strain Sweep 5': 'RGD.PXP.RGD'})

In [22]:
rco_txt = rh.recovery(start, txt, name, rtype=2)
rco_txt = rco_txt.rename(index={'Cyclic Strain Sweep 5': 'T40A'})

In [23]:
frames = [rco_pxp, rco_txt]
rco = pd.concat(frames, sort=False)
rco

,n1 t [s],n2 t [s],n3 t [s],n4 t [s],Mean
Test,,,,,
RGD.PXP.RGD,6.763947,4.976236,16.704571,4.873514,8.329567
T40A,4.520176,4.280487,4.434718,5.000071,4.558863
